## ***Web Scrapping en AWS***

Decidimos hacer web scrapping con AWS Lambda a la página "NYC Taxi & Limousine Commision" para extraer los datasets correspondientes a la información sobre los viajes de los taxis verdes y amarillos realizados durante el año 2023 para luego ingestar estos datos en un bucket de S3.

In [ ]:
import os
import json
import requests
import boto3
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
import chromedriver_binary

# Configurar S3
s3_client = boto3.client('s3')
BUCKET_NAME = 'tu-bucket-de-s3'
FOLDER_NAME = 'carpeta-en-s3'

def lambda_handler(event, context):
    try:
        # Configurar el navegador
        opts = Options()
        opts.add_argument("--headless")
        opts.add_argument("--no-sandbox")
        opts.add_argument("--disable-dev-shm-usage")
        opts.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36")

        driver = webdriver.Chrome(ChromeDriverManager().install(), options=opts)
        wait = WebDriverWait(driver, 10)

        url = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
        driver.get(url)

        tabla_celdas = driver.find_elements(By.XPATH, "//div[@id='faq2023']//table//tr//td")

        def obtenerLinks(celdas):
            for td in celdas:
                listas = td.find_elements(By.TAG_NAME, "ul")
                for elemento_lista in listas:
                    links_columns = elemento_lista.find_elements(By.TAG_NAME, "a")
                    for link in links_columns:
                        titulo = link.get_attribute("title")
                        if titulo == "Yellow Taxi Trip Records" or titulo == "Green Taxi Trip Records":
                            enlace = link.get_attribute("href")
                            nombre_archivo = enlace.split("/")[-1]
                            response = requests.get(enlace, stream=True)
                            if response.status_code == 200:
                                s3_key = f"{FOLDER_NAME}/{nombre_archivo}"
                                s3_client.upload_fileobj(response.raw, BUCKET_NAME, s3_key)
                                print(f"Archivo '{nombre_archivo}' guardado en S3 en '{s3_key}'")
                            else:
                                print(f"Error al descargar el archivo: {enlace}")

        obtenerLinks(tabla_celdas)
        driver.quit()

        return {
            'statusCode': 200, # Indica éxito
            'body': json.dumps('Scraping y almacenamiento en S3 completado')
        }
    except Exception as e:
        return {
            'statusCode': 500,  # Indica error del servidor
            'body': json.dumps(f'Error: {str(e)}')
        }